# Import Required Libraries
Import necessary libraries such as pandas and numpy.

In [16]:
# Import Required Libraries
import pandas as pd
import numpy as np
from collections import defaultdict

# Load and Process Data
Load the provided CSV files (sms3_timetable, sms3_sections, sms3_departments) and process them to extract relevant information for generating enrollment data.

In [17]:
# Load and Process Data
# Load CSV files
timetable_df = pd.read_csv(r'c:\xampp\htdocs\SMS\sms3_timetable.csv')
sections_df = pd.read_csv(r'c:\xampp\htdocs\SMS\sms3_sections.csv')
departments_df = pd.read_csv(r'c:\xampp\htdocs\SMS\sms3_departments.csv')

# Display the first few rows of each dataset for verification
print("Timetable Data:")
print(timetable_df.head())

print("\nSections Data:")
print(sections_df.head())

print("\nDepartments Data:")
print(departments_df.head())

Timetable Data:
   id  subject_id  section_id  room_id day_of_week start_time  end_time
0  36          14           5        1      Monday   06:00:00  08:00:00
1  37           7           5        1      Monday   08:00:00  10:00:00
2  38           8           5        4     Tuesday   06:00:00  08:00:00
3  39           9           5       11     Tuesday   08:00:00  10:00:00
4  41          12           5        1      Friday   08:00:00  11:00:00

Sections Data:
   id  section_number  year_level  department_id branch  capacity  \
0   5            1101           1              1   Main        50   
1   6            1102           1              1   Main        50   
2   7            1103           1              1   Main        50   
3  10            1201           1              1   Main        50   
4  11            1202           1              1   Main        50   

   semester_id  available  
0            1         50  
1            1         50  
2            1         50  
3        

# Generate Enrollment Data
Generate 500k enrollment records with timetable columns based on timetable_id, ensuring the same section_id for a set of timetables. Include NULL for missing timetables, and set receipt_status to 'Paid' and status to 'Approved'.

In [18]:
# Group timetable IDs by section_id
timetable_groups = timetable_df.groupby('section_id')['id'].apply(list).to_dict()

num_records = 500000
max_students = 90000  # Adjust this based on desired number of unique students

# Generate a list of student_ids from 1 to max_students
student_ids = list(range(1, max_students + 1))

enrollment_data = []
student_enrollments = defaultdict(set)  # Tracks (academic_year, semester_id) per student_id

for _ in range(num_records):
    # Randomly select a student_id from the pre-generated list
    student_id = np.random.choice(student_ids)
    
    # Generate random date between January, February, July, and August of each year from 2010 to 2024
    year = np.random.randint(2010, 2025)  # Random year between 2010 and 2024
    month = np.random.choice([1, 2, 7, 8])  # Randomly choose January, February, July, or August
    day = np.random.randint(1, 32)  # Random day in the month
    
    # Determine academic year and semester based on the month
    if month in [7, 8]:
        academic_year = year
        semester_id = 1
    else:
        academic_year = year + 1
        semester_id = 2
    
    # Check if student already enrolled in this academic_year and semester
    key = (academic_year, semester_id)
    if key in student_enrollments[student_id]:
        # If already enrolled, skip this iteration
        continue
    else:
        # Add this enrollment to the student's records
        student_enrollments[student_id].add(key)
    
    # Generate hour, minute, and second for the timestamp
    hour = np.random.randint(6, 18)  # Random hour between 6 AM and 6 PM
    minute = np.random.randint(0, 60)
    second = np.random.randint(0, 60)
    
    try:
        random_date = pd.Timestamp(year=year, month=month, day=day, hour=hour, minute=minute, second=second)
    except ValueError:
        # Handle invalid dates (e.g., February 30)
        random_date = pd.Timestamp(year=year, month=month, day=28, hour=hour, minute=minute, second=second)
    
    # Get the selected section and its timetables
    selected_section = np.random.choice(list(timetable_groups.keys()))
    timetables = timetable_groups[selected_section]
    
    # Prepare timetable columns, ensuring up to 8 timetables, filling with None if necessary
    timetable_columns = timetables[:8] if len(timetables) >= 8 else timetables + [None] * (8 - len(timetables))
    timetable_columns = [int(t) if t is not None else None for t in timetable_columns]
    
    # Create the enrollment record
    enrollment_record = {
        "id": len(enrollment_data) + 1,  # Unique record ID
        "student_id": student_id,
        "timetable_1": timetable_columns[0] if len(timetable_columns) > 0 else None,
        "timetable_2": timetable_columns[1] if len(timetable_columns) > 1 else None,
        "timetable_3": timetable_columns[2] if len(timetable_columns) > 2 else None,
        "timetable_4": timetable_columns[3] if len(timetable_columns) > 3 else None,
        "timetable_5": timetable_columns[4] if len(timetable_columns) > 4 else None,
        "timetable_6": timetable_columns[5] if len(timetable_columns) > 5 else None,
        "timetable_7": timetable_columns[6] if len(timetable_columns) > 6 else None,
        "timetable_8": timetable_columns[7] if len(timetable_columns) > 7 else None,
        "receipt_status": "Paid",
        "status": "Approved",
        "created_at": random_date
    }
    
    enrollment_data.append(enrollment_record)

# Create DataFrame
enrollment_df = pd.DataFrame(enrollment_data)

# Convert timetable columns to Int64 type to handle NULL values
for col in ['timetable_1', 'timetable_2', 'timetable_3', 'timetable_4', 
           'timetable_5', 'timetable_6', 'timetable_7', 'timetable_8']:
    enrollment_df[col] = enrollment_df[col].astype('Int64')

print("Generated Enrollment Data:")
print(enrollment_df.head())

Generated Enrollment Data:
   id  student_id  timetable_1  timetable_2  timetable_3  timetable_4  \
0   1       48766          623          624          625          626   
1   2       17860         7491         7492         7493         7494   
2   3       30187         8203         8204         8205         8206   
3   4       55757         5121         5122         5123         5124   
4   5       52823         5472         5473         5474         <NA>   

   timetable_5  timetable_6  timetable_7  timetable_8 receipt_status  \
0          627          628          629          630           Paid   
1         7495         7496         7497         7498           Paid   
2         8207         8208         8209         8210           Paid   
3         5125         5126         <NA>         <NA>           Paid   
4         <NA>         <NA>         <NA>         <NA>           Paid   

     status          created_at  
0  Approved 2017-08-15 16:48:56  
1  Approved 2018-01-18 11:40:03  

# Save Enrollment Data to CSV
Save the generated enrollment data to a CSV file for further use.

In [19]:
# Save Enrollment Data to CSV
output_filepath = r'c:\\xampp\\htdocs\\SMS\\generated_data_enrollments.csv'
enrollment_df.to_csv(output_filepath, index=False, na_rep='NULL')

print(f"Enrollment data successfully saved to {output_filepath}")

Enrollment data successfully saved to c:\\xampp\\htdocs\\SMS\\generated_data_enrollments.csv
